In [12]:
from langchain.utilities import SQLDatabase

connection_string = 'sqlite:///data/sample.db'
db = SQLDatabase.from_uri(connection_string)
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['maintenance', 'universityconfigurations']


In [7]:
context = db.get_context()
print(list(context))
print(context["table_info"])

['table_info', 'table_names']

CREATE TABLE maintenance (
	"Maintenance_ID" INTEGER, 
	"Base_Station" TEXT NOT NULL, 
	"Maintenance_Date" TEXT NOT NULL, 
	"Technician_Name" TEXT NOT NULL, 
	"Description" TEXT, 
	"Duration_hours" REAL NOT NULL, 
	"Cost" REAL NOT NULL, 
	PRIMARY KEY ("Maintenance_ID"), 
	FOREIGN KEY("Base_Station") REFERENCES universityconfigurations ("Base_Station")
)

/*
3 rows from maintenance table:
Maintenance_ID	Base_Station	Maintenance_Date	Technician_Name	Description	Duration_hours	Cost
1	BS-001	2023-06-15	John Doe	Routine check-up	2.0	150.0
2	BS-002	2023-07-10	Jane Smith	Antenna alignment	3.0	200.0
3	BS-003	2023-07-20	Emily Johnson	Software update	1.5	100.0
*/


CREATE TABLE universityconfigurations (
	"ID" INTEGER NOT NULL, 
	"Base_Station" TEXT NOT NULL, 
	"Location" TEXT NOT NULL, 
	"Bandwidth" REAL NOT NULL, 
	"Transmission_Power" REAL NOT NULL, 
	"Frequency_Band" REAL NOT NULL, 
	"Coverage_Area" REAL NOT NULL, 
	"Maximum_Capacity" INTEGER NOT NULL, 
	"Opera

In [13]:
from langchain_community.chat_models import ChatOllama

chat_model = ChatOllama(
    model="deepseek-r1:1.5b",  
    temperature=0.7,
    base_url="http://localhost:11434",
    # format="json",
)

response = chat_model.invoke("Explain what SQL injection is in one sentence.")
print("Basic response:\n", response)

Basic response:
 content='<think>\nOkay, so I need to explain what SQL injection is in one sentence. Hmm, let me think about this.\n\nFirst, what is SQL? Oh right, it\'s SQL stands for Structured Query Language, which is a programming language used to manipulate data in relational databases. So when they say "SQL injection," that must be something malicious happening on the database level because if someone can get SQL into an application, they could cause issues like unauthorized access or data corruption.\n\nWait, but how exactly does it work? I remember that SQL injection occurs when a script or piece of code executes arbitrary SQL commands. These commands are then injected into a web page, server, or other application, causing unexpected behavior or errors. For example, if an attacker injects SQL commands that modify user credentials or navigate to malicious sites.\n\nI should make sure the explanation is clear and concise. So putting it all together: SQL injection is when arbitrar

In [14]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content='''You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the 
input question.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".
If mot mentioned do not use {top_k} rows. Output only the SQL Query without any output or result of the query.
If the answer could not be found, give a single line meaningful error'''),
    HumanMessage(content="Count the number of base stations?")
]

response = chat_model.invoke(messages)
print("Response with context:\n", response)

Response with context:
 content='<think>\nAlright, so I need to figure out how many base stations are in the database. Let me start by recalling what I know about SQLite queries. The question mentions using only specific columns and not querying all of them. \n\nI remember that in my previous questions, when someone asked "Count the number of base stations?", the response involved a query that selects a count from the station table. The SQL was `SELECT COUNT(``base stations ```) as count FROM station`. So, I should follow a similar structure here.\n\nFirst, I\'ll need to select just one column which is the count of the \'base stations\' field. Since it\'s stored in a column that might be quoted or not, but in SQLite, when you use double quotes around a string, they are treated as literals unless there\'s an escape character. However, since we\'re selecting the count directly, I can just write `COUNT(``base stations ```)`.\n\nNow, considering the tables provided earlier, which one conta

'<think>\nOkay, so I need to figure out how to answer this question using SQLite. The user provided some context about two tables: maintenance and universityconfigurations. They also gave me a table of maintenance records with their details.\n\nThe question is asking for the station ID. Looking at the maintenance table, each row has a "Maintenance_ID" which seems to be unique per base station. The base station name is in the format BS-00X, where X is the station number. For example, BS-001, BS-002, etc.\n\nSo, if I want to find out the station ID for a specific base station, I can join the maintenance table with the universityconfigurations table on "Base_Station". That way, I can get both the maintenance_id and the corresponding base station name from the universityconfigurations table.\n\nI should write an SQL query that selects the maintenance_id. But since each maintenance_id corresponds to a unique base station, I need to make sure that the query only returns the maintenance_id fo